# Property Price Index Analysis

About Property Value and Investment Indexes

Note that there is no standard formula to calculate property price indexes. The formulas differ from Case-Shiller Index, UK Housing Price Index, etc.

**Price to Income Ratio** is a fundamental measure for apartment purchase affordability, where a lower ratio indicates better affordability. It is typically calculated as the ratio of median apartment prices to median familial disposable income, expressed as years of income (although variations are used also elsewhere). Our formula assumes and uses:

- net disposable family income, as defined as 1.5 * the average net salary (50% is the assumed percentage of women in the workforce)
- median apartment size is 90 square meters
- price per square meter (the formula uses) is the average price of square meter in the city center and outside of the city center

**Mortgage as Percentage of Income** is a ratio of the actual monthly cost of the mortgage to take-home family income (lower is better). The average monthly salary is used to estimate family income. It assumes a 100% mortgage is taken on 20 years for the house(or apt) of 90 square meters which price per square meter is the average of prices in the city center and outside of the city center.

**Loan Affordability Index** is an inverse of mortgage as percentage of income. The used formula is : `(100 / mortgage as percentage of income)` (higher is better).

**Price to Rent Ratio** is the average cost of ownership divided by the received rent income (if buying to let) or the estimated rent that would be paid if renting (if buying to reside). Lower values suggest that it is better to buy rather than rent, and higher values suggest that it is better to rent rather than buy. Our formula to estimate rent per square meter assumes 1 bedroom apt has 50 square meters and 3 bedroom apartment has 110 square meters. It doesn't take into account taxes or maintenance fees.

**Gross Rental Yield** is the total yearly gross rent divided by the house price (expressed in percentages). Higher is better.

In [8]:
import pandas as pd
import json

# Set display option to show all rows
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [9]:
property_price_df = pd.read_json('data/property_price.json')


In [10]:
crime_rate_df = pd.read_json("data/crime_rates.json")

In [14]:
cost_of_living_df = pd.read_json("data/cost_of_living.json")
cost_of_living_index = cost_of_living_df.drop(columns=["Cost of Living Details"])

In [15]:
cost_of_living_index.head()

,Year,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Index,Local Purchasing Power Index
0,2024,"Bern, Switzerland",110.5,42.3,78.2,108.3,96.5,130.7
1,2024,"San Jose, CA, United States",90.5,72.2,81.8,90.4,83.5,139.7
2,2024,"New York, NY, United States",100.0,100.0,100.0,100.0,100.0,100.0
3,2024,"Geneva, Switzerland",109.8,66.8,89.4,114.0,107.3,112.2
4,2024,"Lausanne, Switzerland",117.7,46.4,83.9,123.3,106.2,106.7


In [16]:
merged_df = pd.merge(property_price_df, crime_rate_df, on=['City', 'Year'], how='left')
merged_df = pd.merge(merged_df, cost_of_living_index, on=["City", "Year"], how="left")

In [17]:
united_states_canada_only = merged_df[merged_df["City"].str.contains("United States") | merged_df["City"].str.contains("Canada")]

In [18]:
yearly_property_price_df = {}

for year in united_states_canada_only["Year"].unique():
    yearly_property_price_df[year] = (
        united_states_canada_only[united_states_canada_only["Year"] == year]
        .sort_values(by="Price to Income Ratio", ascending=True)
        .reset_index(drop=True)
    )

In [19]:
yearly_property_price_df["2024-mid"].head(20)

,Year,City,Price to Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside Centre,Price to Rent Ratio City Centre,Price to Rent Ratio Outside City Centre,Mortgage as a Percentage of Income,Affordability Index,Crime Index,Safety Index,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Index,Local Purchasing Power Index
0,2024-mid,"Detroit, MI, United States",1.2,32.9,21.8,3.0,4.6,10.9,9.2,74.3,25.7,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-mid,"Rochester, NY, United States",1.8,23.0,20.4,4.3,4.9,16.1,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-mid,"Indianapolis, IN, United States",1.8,16.1,14.8,6.2,6.7,16.6,6.0,60.2,39.8,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-mid,"Memphis, TN, United States",1.9,16.4,15.9,6.1,6.3,17.0,5.9,76.5,23.5,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-mid,"Milwaukee, WI, United States",2.0,20.0,16.8,5.0,6.0,17.8,5.6,67.6,32.4,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-mid,"Omaha, NE, United States",2.2,17.9,13.6,5.6,7.4,20.1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-mid,"Baltimore, MD, United States",2.3,18.6,16.6,5.4,6.0,21.1,4.7,74.7,25.3,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-mid,"Tulsa, OK, United States",2.3,13.7,11.5,7.3,8.7,20.5,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-mid,"Atlanta, GA, United States",2.4,12.9,12.6,7.7,7.9,22.0,4.6,64.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-mid,"Houston, TX, United States",2.5,12.6,13.7,7.9,7.3,22.1,4.5,63.6,36.4,65.1,39.2,52.7,66.8,60.7,162.9


# Plot the data into a graph

In [20]:
def plot_graph(df, city):
    fig = make_subplots(rows=4, cols=4, start_cell="top-left", subplot_titles=df.columns[1:])
    fig.update_layout(title=city, height=1500, width= 1500)
    curr_row = 1
    curr_col = 1
    for col in df.columns[1:]:
        fig.add_trace(
            go.Scatter(x=df["Year"], y=df[col], mode="lines+markers", name=col),
            row=curr_row,
            col=curr_col,
        )
        fig.update_xaxes(title_text="Year", row=curr_row, col=curr_col)
        fig.update_yaxes(title_text=col, row=curr_row, col=curr_col)
        curr_col += 1
        if curr_col == 5:
            curr_row += 1
            curr_col = 1
            
    fig.show()
    pio.write_image(fig, f"plots/{city}.png")

In [25]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

cities = yearly_property_price_df["2024-mid"].head(10)['City'].tolist() + ['Toronto, Canada', "New York, NY, United States", "Vancouver, Canada"]

In [26]:
for city in cities:
    selected_city_df = merged_df[merged_df["City"] == city].sort_values(
        "Year", ascending=True
    )
    selected_city_df.drop(columns=["City"], inplace=True)
    plot_graph(selected_city_df, city)

As an investor:
- Price to income ratio:
    - Higher the ratio:
        - Potential appreciation: High price-to-income ratios often occur in desirable areas with strong demand (e.g., large cities), which could lead to property appreciation over time.
        - Limited rental pool: The higher prices may mean fewer local residents can afford to buy homes, increasing the demand for rentals. However, if rents can't rise with property prices, this could limit your rental income.
        - Risk of market correction: In extreme cases, a high ratio can signal a housing bubble, increasing the risk of price drops in the future.
    - Lower the ratio:
        - Limited appreciation: These markets may not see as much property value appreciation compared to high-ratio markets, making it better for investors focused on income rather than capital gains
        - Better cash flow: Properties with lower price-to-income ratios might be more affordable to purchase and could still offer strong rental income. This leads to better cash flow.
        - Steadier rental demand: In markets with a low ratio, more people may be able to afford to rent or buy, which could keep rental demand steady.
- Price to rent ratio:
    - The higher the ratio: indicates that property prices are rising faster than rents
    - The lower the ratio: Generally more favorable for investors focused on cash flow, as you’re getting more rental income for the price of the property.
- Gross rental yield:
    - Higher the yield: The better the cash flow
    - Lower the yield: The lower the cash flow
    - Industry Benchmark: A gross rental yield of 5-8% is often considered reasonable for residential properties, though this can vary widely by market. For example, urban areas with high appreciation potential may have yields under 5%, while properties in more affordable or emerging markets may have yields above 8%.
- Mortgage as a percentage of income:
    - Low Mortgage-to-Income Ratio:
        - More cash flow, less risk, less leverage
    - High mortgage-to-income ratio:
        - Less cash flow, more risk, more leverage